In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split
from datetime import datetime

torch.manual_seed(123)
torch.set_default_dtype(torch.double)

In [17]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))
print(device)

cuda


# 3.1
Load and preprocess the CIFAR-10 dataset. Split it into 3 datasets: training, validation and
test. Take a subset of these datasets by keeping only 2 labels: bird and plane.

In [18]:
def load_cifar(train_val_split=0.9, data_path='../data/', preprocessor=None):
    
    # Define preprocessor if not already given
    if preprocessor is None:
        preprocessor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4915, 0.4823, 0.4468),
                                (0.2470, 0.2435, 0.2616))
        ])
    
    # load datasets
    data_train_val = datasets.CIFAR10(
        data_path,      
        train=True,      
        download=True,
        transform=preprocessor)

    data_test = datasets.CIFAR10(
        data_path, 
        train=False,
        download=True,
        transform=preprocessor)

    # train/validation split
    n_train = int(len(data_train_val)*train_val_split)
    n_val =  len(data_train_val) - n_train

    data_train, data_val = random_split(
        data_train_val, 
        [n_train, n_val],
        generator=torch.Generator().manual_seed(123)
    )

    print("Size of the train dataset:        ", len(data_train))
    print("Size of the validation dataset:   ", len(data_val))
    print("Size of the test dataset:         ", len(data_test))
    
    return (data_train, data_val, data_test)

cifar10_train, cifar10_val, cifar10_test = load_cifar()

# Now define a lighter version of CIFAR10: cifar
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']

# For each dataset, keep only airplanes and birds
cifar2_train = [(img, label_map[label]) for img, label in cifar10_train if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]
cifar2_test = [(img, label_map[label]) for img, label in cifar10_test if label in [0, 2]]

print('Size of the training dataset: ', len(cifar2_train))
print('Size of the validation dataset: ', len(cifar2_val))
print('Size of the test dataset: ', len(cifar2_test))

Files already downloaded and verified
Files already downloaded and verified
Size of the train dataset:         45000
Size of the validation dataset:    5000
Size of the test dataset:          10000
Size of the training dataset:  9017
Size of the validation dataset:  983
Size of the test dataset:  2000


# 3.2
Write a MyMLP class that implements a MLP in PyTorch (so only fully connected layers) such
that:
(a) The input dimension is 3072 (= 32*32*3) and the output dimension is 2 (for the 2
classes).
(b) The hidden layers have respectively 512, 128 and 32 hidden units.
(c) All activation functions are ReLU. The last layer has no activation function since the
cross-entropy loss already includes a softmax activation function.

In [19]:
class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()  

        self.flat = nn.Flatten()
        # 32*32*3: determined by our dataset: 32x32 RGB images
        self.fc1 = nn.Linear(32*32*3, 512)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 128)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 32)
        self.act3 = nn.ReLU()
        # 2: determined by our number of classes (birds and planes)
        self.fc4 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.flat(x)
        out = self.act1(self.fc1(out))
        out = self.act2(self.fc2(out))
        out = self.act3(self.fc3(out))
        out = self.fc4(out)
        return out

# 3.3
Write a train(n epochs, optimizer, model, loss fn, train loader) function that trains
model for n epochs epochs given an optimizer optimizer, a loss function loss fn and a dat-
aloader train loader.

In [20]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    n_batch = len(train_loader)
    losses_train = []
    model.train()
    optimizer.zero_grad(set_to_none=True)
    
    for epoch in range(1, n_epochs + 1):
        
        loss_train = 0.0
        for imgs, labels in train_loader:

            imgs = imgs.to(device=device, dtype=torch.double)
            labels = labels.to(device=device)

            outputs = model(imgs)
            
            loss = loss_fn(outputs, labels)
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()

            loss_train += loss.item()
            
        losses_train.append(loss_train / n_batch)

        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
    return losses_train

# 3.4
Write a similar function train manual update that has no optimizer parameter, but a learn-
ing rate lr parameter instead and that manually updates each trainable parameter of model
using equation (3). Do not forget to zero out all gradients after each iteration.

In [21]:
def train_manual_update(n_epochs, lr, model, loss_fn, train_loader, verbose = False):
    n_batch = len(train_loader)
    losses_train = []
    model.train()
    
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:

            imgs = imgs.to(device=device, dtype=torch.double)
            labels = labels.to(device=device)

            outputs = model(imgs)
            
            loss = loss_fn(outputs, labels)
            loss.backward()
            
            # optimizer step
            with torch.no_grad():
                for p in model.parameters():
                    p -= lr * p.grad
            
            # zero out all gradients
            with torch.no_grad():
                for p in model.parameters():
                    p.grad.zero_()

            loss_train += loss.item()
            
        losses_train.append(loss_train / n_batch)

        if epoch == 1 or epoch % 10 == 0 or verbose:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
    return losses_train

# 3.5
Train 2 instances of MyMLP, one using train and the other using train manual update (use
the same parameter values for both models). Compare their respective training losses.

In [22]:
torch.manual_seed(123)
model = MyMLP().to(device=device) 
optimizer = optim.SGD(model.parameters(), lr=1e-2)
train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
n_epochs = 10
train(n_epochs, optimizer, model, nn.CrossEntropyLoss(), train_loader)

11:02:36.585104  |  Epoch 1  |  Training loss 0.645
11:02:46.574501  |  Epoch 10  |  Training loss 0.326


[0.6451217197440591,
 0.5329839468263902,
 0.4778253794609127,
 0.4468125583284399,
 0.4197904609343939,
 0.3968746333461311,
 0.3773816702722962,
 0.3595321877492997,
 0.3426136703937787,
 0.32593414376821933]

In [23]:
torch.manual_seed(123)
model = MyMLP().to(device=device) 
train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
n_epochs = 10
train_manual_update(n_epochs, 1e-2, model, nn.CrossEntropyLoss(), train_loader)

11:02:47.830246  |  Epoch 1  |  Training loss 0.645
11:02:58.042466  |  Epoch 10  |  Training loss 0.326


[0.645121719744059,
 0.5329839468263902,
 0.4778253794609127,
 0.44681255832843997,
 0.4197904609343939,
 0.3968746333461311,
 0.3773816702722962,
 0.3595321877492997,
 0.34261367039377877,
 0.3259341437682193]

We see that the train and train_manual_update function produces the same training losses when given the same data and learning rate.

# 3.6

Modify train manual update by adding a L2 regularization term in your manual parameter
update. Add an additional weight decay parameter to train manual update. Compare
again train and train manual update results with 0 < weight decay < 1

# 3.7

Modify train manual update by adding a momentum term in your parameter update. Add
an additional momentum parameter to train manual update. Check again the correctness of
the new update rule by comparing it to train function (with 0 < momentum < 1).

# 3.8

Train different instances (at least 4) of the MyMLP model with different learning rate, momentum
and weight decay values . You can choose the same values as in the
gradient descent output.txt file

# 3.9
Select the best model among those trained in the previous question based on their accuracy

# 3.10
Evaluate the best model